# Importação e visualização do banco de dados

**1. Importar as bibliotecas que irão ser utilizadas:**

In [62]:
import numpy as np #a. Numpy as np
import pandas as pd #b. Pandas as pd
import matplotlib.pyplot as plt #c. Matplotlib.pyplot
import seaborn as sns #d. seaborne as sns
%matplotlib inline
np.random.seed(0) #f. Configurar o random seed para 0 (np.random.seed(0))

**2. Transformar arquivos em dataframe e gerar momentos estatísticos**

In [63]:
#a. Importar os arquivos “kaggle/input/titanic/test.csv” e “kaggle/input/titanic/train.csv”
titanic_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
#b. Gerar momentos estatísticos sobre o dataframe de treino, utilizando “.describe( )”
titanic_data.describe(include="all")

**3. Análise dos dados**

In [64]:
#a. Identificar as Features do banco de dados que são categóricas (não numéricas), e as que são numéricas. As numéricas, deve-se separá-las em discretas e contínuas.
#  i. Pode-se usar o método “.info( )” para descobrir o tipo de dado de cada Feature.
print(test_data.info())
test_data.sample(5)

In [65]:
#b. Descobrir quais as Features contém NaN, além da quantidade desse tipo de dado em cada Feature.
#  i. Pode-se utilizar o comando “DataFrame.isnull.sum( )”
print(pd.isnull(titanic_data).sum())

In [66]:
#c. Deve-se utilizar apenas Features numéricas e retirar as categóricas. Portanto, faça dois novos dataframes, apenas com as features numéricas do banco de dados de treino e de teste.
titanic_data_num = titanic_data[['PassengerId','Age', 'Fare', 'SibSp', 'Parch', 'Survived']]
test_data_num = test_data[['PassengerId','Age', 'Fare', 'SibSp', 'Parch']]

#d. Retire as linhas que contenham NaN das features do banco de dados de treino.
#  i. Para isso será útil o método DataFrame.dropna( )
titanic_data_num = titanic_data_num.dropna( )

#e. no banco de dados de teste, substitua os NaN pelo valor 0, para que o arquivo de predição tenha o número de linhas exigido pela competição
#  i. Utilizar o método DataFrame.fillna( )
test_data_num = test_data_num.fillna(0)

#  Exploração do banco de dados

**1. Para a Feature “Age” será necessário criar uma nova Feature que a separe em 8 grupos para que se possa utilizar “sns.barplot”. (DICA: Crie um novo dataframe apenas com as informações necessárias para a criação deste gráfico)**


In [67]:
#a. Criar uma lista com os nomes que serão dados aos grupos em que as idades serão repartidas (Exemplo: “(0-10)”, “(10-20)”, ...)
bins = [0, 5, 10, 15, 20, 25, 35, 45, 60, np.inf]
labels = ['(0-5)', '(5-10)', '(10-15)', '(15-20)', '(20-25)', '(25-35)', '(35-45)','(45-60)', 'Idoso']

#b. utilize pandas.cut, para separar as idades em 8 grupos e utilize o argumento “labels” para utilizar a lista dos nomes dos grupos
#c. Criar a nova coluna com os grupos, perceba que esta nova coluna será categórica
titanic_data_num['AgeGroup'] = pd.cut(titanic_data_num["Age"], bins, labels = labels)
test_data_num['AgeGroup'] = pd.cut(test_data_num["Age"], bins, labels = labels)

#d. Criar um gráfico que indique a chance de sobrevivência por grupo de idade
sns.barplot(x="AgeGroup", y="Survived", data=titanic_data_num)
plt.show()

#e. Faça uma análise a partir da visualização deste gráfico

Crianças de 0-5 anos tem a mais alta taxa de sobrevivencia enquanto idosos tem a menor taxa de sobrevivencia, o que é esperado. As taxas de sobrevivencia dos outros grupos são contra intuitivas.

**2. Crie gráficos para todas as outras features numéricas discretas, e faça uma análise dos resultados obtidos. Não será necessário produzir um gráfico para a feature “Fare”. Após a criação de cada gráfico será necessário realizar uma breve análise para avaliação do impacto de cada feature na predição**

In [68]:
sns.barplot(x="Parch", y="Survived", data=titanic_data_num)

In [69]:
sns.barplot(x="SibSp", y="Survived", data=titanic_data_num)

# Treinamento do modelo e submissão das predições

**1. Separação de banco de dados de treino e validação.**

In [70]:
#a. Criar variável X, contendo o banco de dados apenas das Features
X = titanic_data_num.drop(['PassengerId','Survived', 'AgeGroup'], axis=1)

#b. Criar variável y, contendo o target da predição
y = titanic_data_num["Survived"]

#c. Usar Train Test Split para separar treino e validação (DICA: utilizar validação com 20% do tamanho do treino)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

**2. Treinamento e validação do modelo**

In [71]:
#a. Importar Random Forest Classifier da biblioteca sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier

#b. Importar accuracy score da biblioteca sklearn.metrics
from sklearn.metrics import accuracy_score

#c. Treinar random forest com as features e o target de treino
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)

#d. Realizar a predição
y_pred = randomforest.predict(x_val)

#e. Validar a predição, comparando com o target da validação, utilizando accuracy score
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

#f. Avalie se a pontuação é satisfatória


Maior que 50%, mas com certeza da pra melhorar

**3. Gerar predição, salvar a versão e submeter à competição**

In [72]:
#a. Treinar novamente o modelo, utilizando o banco de dados completo
randomforest = RandomForestClassifier()
randomforest.fit(X, y)

#b. Salvar o índice do banco de dados de teste
ids = test_data_num['PassengerId']

#c. Gerar uma predição a partir do modelo, sendo a entrar o banco de dados de teste, sem seu índice
predictions = randomforest.predict(test_data_num.drop(['PassengerId','AgeGroup'], axis=1))

#d. Criar um dataframe com o pandas que contenha: os mesmos índices do banco de dados de teste, as predições realizadas (OBS: Olhar arquivo modelo "gender_submission.csv")
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })

#e. Transformar este dataframe em um arquivo csv utilizando pandas
output.to_csv('submission.csv', index=False)

#f. Salvar a versão clicando em “Save Version”, selecionar a versão que foi salva e submeter à competição